In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
import pandas as pd
import re
import time
import chromedriver_autoinstaller
chromedriver_autoinstaller.install
driver=webdriver.Chrome()
website='https://www.drdata.in/list-doctors.php'
driver.get(website)
#opened the website using the driver
driver.maximize_window()
#now finding the categories of the doctors we have available
search_bar=driver.find_element(By.ID,'specialization')
options=search_bar.find_elements(By.TAG_NAME,'option')
ops=[]
#_________________________________list  of the doctors available, is stored in ops___________________________________________
for opts in options:
    ops.append(opts.text)
ops.pop(0)
#_____________________________________________end here_____________________________________________________
#____________________________________________find pages___________________________________________________
def find_pages(pg):
    nops=(pg//15)+1
    return nops
#_____________________________________________now iterating the doctors list_______________________________________________________
for item in (ops):
        name=[]
        speciality=[]
        degree=[]
        state=[]
        city=[]
        more_details=[]
        doctor_name=item
        base_url='https://www.drdata.in/list-doctors.php?specialization='
        url=base_url+doctor_name+"&page"+str(1)
        driver.get(url)
        time.sleep(3)
        div=driver.find_element(By.XPATH,'/html/body/div[1]/div[3]')
        num=div.find_element(By.XPATH,'./h1/span').text
        num=int(num)
        nops=find_pages(num)
        if(nops>=11):
            print(f' Scrape of {doctor_name}  has  skipped as it has {num} doctors and {nops} pages')
            continue  
        print(f' {num} {doctor_name} doctors and has {nops} pages of data')
        i=0
        while((i+1)<=nops):
            time.sleep(3)
            table=driver.find_element(By.XPATH,"//table[@class='table-bordered table-striped table-condensed cf']")
            tbody=table.find_element(By.XPATH,'./tbody')
            trs=tbody.find_elements(By.XPATH,'./tr')
            for tr in trs:
                name.append(tr.find_element(By.XPATH,'./td[1]').text)
                speciality.append(tr.find_element(By.XPATH,'./td[2]').text)
                degree.append(tr.find_element(By.XPATH,'./td[3]').text)
                state.append(tr.find_element(By.XPATH,'./td[4]').text)
                city.append((tr.find_element(By.XPATH,'./td[5]').text))
                more=tr.find_element(By.XPATH,'./td[6]')
                more_details.append(more.find_element(By.XPATH,'./a').get_attribute('href'))
            i=i+1
            new_url=base_url+doctor_name+"&page="+str(i+1)
            driver.get(new_url)
            trs.clear()
        data=pd.DataFrame({"Name":name,"Speciality":speciality,"Degree":degree,"State":state,"City":city,"More Details":more_details})
        data.to_csv(f'{doctor_name}.csv',index=False)
time.sleep(3)
driver.quit()
            
